In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings("ignore")

In [2]:
print("Step 1: Loading raw datasets...")
matches = pd.read_csv('matches.csv')
deliveries = pd.read_csv('deliveries.csv')

Step 1: Loading raw datasets...


In [3]:
print("\nStep 2: Initial data exploration...")
print("\nMatches dataset info:")
matches.info()
print("\nMatches dataset summary statistics:")
print(matches.describe())
print("\nNull values in matches dataset:")
print(matches.isnull().sum())


Step 2: Initial data exploration...

Matches dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 980 entries, 0 to 979
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               980 non-null    int64  
 1   season           980 non-null    int64  
 2   city             929 non-null    object 
 3   date             980 non-null    object 
 4   match_type       980 non-null    object 
 5   player_of_match  976 non-null    object 
 6   venue            980 non-null    object 
 7   team1            980 non-null    object 
 8   team2            980 non-null    object 
 9   toss_winner      980 non-null    object 
 10  toss_decision    980 non-null    object 
 11  winner           976 non-null    object 
 12  result           980 non-null    object 
 13  result_margin    963 non-null    float64
 14  target_runs      978 non-null    float64
 15  target_overs     978 non-null    float64
dtypes:

In [ ]:
print("\nStep 3: Removing unnecessary columns...")
# Remove columns that aren't needed for analysis
matches.drop(['umpire1', 'umpire2', 'method', 'super_over'], axis=1, inplace=True)
print("Removed columns: umpire1, umpire2, method, super_over")

In [5]:
print("\nStep 4: Standardizing team names...")
# Create a mapping dictionary for team name standardization
team_name_mapping = {
    'Royal Challengers Bangalore': 'Royal Challengers Bengaluru',
    'Kings XI Punjab': 'Punjab Kings',
    'Delhi Daredevils': 'Delhi Capitals',
    'Deccan Chargers': 'Sunrisers Hyderabad',
    'Rising Pune Supergiants': 'No longer active',
    'Rising Pune Supergiant': 'No longer active',
    'Kochi Tuskers Kerala': 'No longer active',
    'Pune Warriors': 'No longer active',
    'Gujarat Lions': 'No longer active',
}


Step 4: Standardizing team names...


In [6]:
for col in ['team1', 'team2', 'toss_winner', 'winner']:
    matches[col] = matches[col].replace(team_name_mapping)

# Update team names in deliveries dataset
for col in ['batting_team', 'bowling_team']:
    deliveries[col] = deliveries[col].replace(team_name_mapping)


In [7]:
print("\nStep 5: Filtering out inactive teams...")
inactive_teams = ['No longer active']
matches = matches[~matches['team1'].isin(inactive_teams)]
matches = matches[~matches['team2'].isin(inactive_teams)]
deliveries = deliveries[~deliveries['batting_team'].isin(inactive_teams)]
deliveries = deliveries[~deliveries['bowling_team'].isin(inactive_teams)]


Step 5: Filtering out inactive teams...


In [8]:
teams_in_matches = pd.unique(matches[['team1', 'team2']].values.ravel('K'))
print("Teams in matches dataset after standardization:")
print(sorted(teams_in_matches))

teams_in_deliveries = pd.unique(deliveries[['batting_team', 'bowling_team']].values.ravel('K'))
print("Teams in deliveries dataset after standardization:")
print(sorted(teams_in_deliveries))

Teams in matches dataset after standardization:
['Chennai Super Kings', 'Delhi Capitals', 'Gujarat Titans', 'Kolkata Knight Riders', 'Lucknow Super Giants', 'Mumbai Indians', 'Punjab Kings', 'Rajasthan Royals', 'Royal Challengers Bengaluru', 'Sunrisers Hyderabad']
Teams in deliveries dataset after standardization:
['Chennai Super Kings', 'Delhi Capitals', 'Gujarat Titans', 'Kolkata Knight Riders', 'Lucknow Super Giants', 'Mumbai Indians', 'Punjab Kings', 'Rajasthan Royals', 'Royal Challengers Bengaluru', 'Sunrisers Hyderabad']


In [9]:
print("\nStep 6: Preparing for dataset merge...")
# Rename 'id' column in matches to 'match_id' for consistent naming with deliveries
matches.rename(columns={'id': 'match_id'}, inplace=True)


Step 6: Preparing for dataset merge...


In [10]:
print("\nStep 7: Merging datasets...")
# Merge datasets to create ball-by-ball data with match information
merged_df = deliveries.merge(matches, on='match_id', how='left')


Step 7: Merging datasets...


In [11]:
merged_df.drop(['team1', 'team2'], axis=1, inplace=True)

In [14]:
print("\nStep 8: Handling missing values...")
venue_city_map = {
    "M Chinnaswamy Stadium": "Bengaluru",
    "Punjab Cricket Association Stadium, Mohali": "Mohali",
    "Feroz Shah Kotla": "Delhi",
    "Wankhede Stadium": "Mumbai",
    "Eden Gardens": "Kolkata",
    "Sawai Mansingh Stadium": "Jaipur",
    "Rajiv Gandhi International Stadium, Uppal": "Hyderabad",
    "MA Chidambaram Stadium, Chepauk": "Chennai",
    "Dr DY Patil Sports Academy": "Mumbai",
    "Newlands": "Cape Town",
    "St George's Park": "Port Elizabeth",
    "Kingsmead": "Durban",
    "SuperSport Park": "Centurion",
    "Buffalo Park": "East London",
    "New Wanderers Stadium": "Johannesburg",
    "De Beers Diamond Oval": "Kimberley",
    "OUTsurance Oval": "Bloemfontein",
    "Brabourne Stadium": "Mumbai",
    "Sardar Patel Stadium, Motera": "Ahmedabad",
    "Barabati Stadium": "Cuttack",
    "Brabourne Stadium, Mumbai": "Mumbai",
    "Vidarbha Cricket Association Stadium, Jamtha": "Nagpur",
    "Himachal Pradesh Cricket Association Stadium": "Dharamshala",
    "Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium": "Visakhapatnam",
    "Subrata Roy Sahara Stadium": "Pune",
    "Shaheed Veer Narayan Singh International Stadium": "Raipur",
    "JSCA International Stadium Complex": "Ranchi",
    "Sheikh Zayed Stadium": "Abu Dhabi",
    "Sharjah Cricket Stadium": "Sharjah",
    "Dubai International Cricket Stadium": "Dubai",
    "Maharashtra Cricket Association Stadium": "Pune",
}


Step 8: Handling missing values...


In [15]:
# Fill missing city values using venue information
merged_df['city'] = merged_df['city'].fillna(merged_df['venue'].map(venue_city_map))

In [ ]:
# Handle other missing values
print("\nStep 9: Handling remaining missing values...")
# Identify matches with missing winners (likely abandoned/tied)
problematic_matches = merged_df[merged_df['winner'].isna()]['match_id'].unique()
print(f"Number of matches with missing winners: {len(problematic_matches)}")



Step 9: Handling remaining missing values...
Number of matches with missing winners: 4


KeyError: 'extras'

In [17]:
if 'extras' in merged_df.columns:
    merged_df['has_extras'] = merged_df['extras'].notna() & (merged_df['extras'] > 0)
elif 'extra_runs' in merged_df.columns:
    merged_df['has_extras'] = merged_df['extra_runs'].notna() & (merged_df['extra_runs'] > 0)
elif 'extras_type' in merged_df.columns:
    merged_df['has_extras'] = merged_df['extras_type'].notna()
else:
    print("Note: No extras column found. Skipping extras indicator creation.")

# For wickets, check if related columns exist
if 'player_dismissed' in merged_df.columns:
    merged_df['is_wicket_binary'] = merged_df['player_dismissed'].notna().astype(int)
elif 'is_wicket' in merged_df.columns:
    merged_df['is_wicket_binary'] = merged_df['is_wicket'].fillna(0).astype(int)
else:
    print("Note: No wicket column found. Skipping wicket indicator creation.")

# Remove matches with missing result_margin (optional, depending on analysis needs)
clean_merged_df = merged_df.dropna(subset=['result_margin'])

# Final null check
print("\nRemaining null values in cleaned dataset:")
print(clean_merged_df.isnull().sum())


Remaining null values in cleaned dataset:
match_id                 0
inning                   0
batting_team             0
bowling_team             0
over                     0
ball                     0
batter                   0
bowler                   0
non_striker              0
batsman_runs             0
extra_runs               0
total_runs               0
extras_type         217524
is_wicket                0
player_dismissed    218693
dismissal_kind      218693
fielder             221842
season                   0
city                     0
date                     0
match_type               0
player_of_match          0
venue                    0
toss_winner              0
toss_decision            0
winner                   0
result                   0
result_margin            0
target_runs              0
target_overs             0
has_extras               0
is_wicket_binary         0
dtype: int64


In [18]:
#Save the cleaned and merged dataset
print("\nStep 10: Saving cleaned dataset...")
clean_merged_df.to_csv("IPL_DATASET.csv", index=False)
print("Cleaned and merged data has been saved to IPL_DATASET.csv")


Step 10: Saving cleaned dataset...
Cleaned and merged data has been saved to IPL_DATASET.csv


In [19]:
print("\nStep 11: Verifying cleaned dataset...")
ipl_data = pd.read_csv("IPL_DATASET.csv")
print("Cleaned dataset shape:", ipl_data.shape)
print("\nCleaned dataset info:")
ipl_data.info()

print("\nData cleaning process completed successfully!")


Step 11: Verifying cleaned dataset...
Cleaned dataset shape: (230103, 32)

Cleaned dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230103 entries, 0 to 230102
Data columns (total 32 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   match_id          230103 non-null  int64  
 1   inning            230103 non-null  int64  
 2   batting_team      230103 non-null  object 
 3   bowling_team      230103 non-null  object 
 4   over              230103 non-null  int64  
 5   ball              230103 non-null  int64  
 6   batter            230103 non-null  object 
 7   bowler            230103 non-null  object 
 8   non_striker       230103 non-null  object 
 9   batsman_runs      230103 non-null  int64  
 10  extra_runs        230103 non-null  int64  
 11  total_runs        230103 non-null  int64  
 12  extras_type       12579 non-null   object 
 13  is_wicket         230103 non-null  int64  
 14  player_dismissed  

In [20]:
print(ipl_data.isnull().sum())

match_id                 0
inning                   0
batting_team             0
bowling_team             0
over                     0
ball                     0
batter                   0
bowler                   0
non_striker              0
batsman_runs             0
extra_runs               0
total_runs               0
extras_type         217524
is_wicket                0
player_dismissed    218693
dismissal_kind      218693
fielder             221842
season                   0
city                     0
date                     0
match_type               0
player_of_match          0
venue                    0
toss_winner              0
toss_decision            0
winner                   0
result                   0
result_margin            0
target_runs              0
target_overs             0
has_extras               0
is_wicket_binary         0
dtype: int64


*Data Preparation - Aggregating to Match Level*

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [23]:
df = pd.read_csv("IPL_DATASET.csv")
df.isnull().sum()

match_id                 0
inning                   0
batting_team             0
bowling_team             0
over                     0
ball                     0
batter                   0
bowler                   0
non_striker              0
batsman_runs             0
extra_runs               0
total_runs               0
extras_type         217524
is_wicket                0
player_dismissed    218693
dismissal_kind      218693
fielder             221842
season                   0
city                     0
date                     0
match_type               0
player_of_match          0
venue                    0
toss_winner              0
toss_decision            0
winner                   0
result                   0
result_margin            0
target_runs              0
target_overs             0
has_extras               0
is_wicket_binary         0
dtype: int64

In [24]:
# Converting 'date' to datetime objects for potential time-based analysis or sorting
df['date'] = pd.to_datetime(df['date'], format='%d-%m-%Y')

In [27]:
# Dropping columns not relevant for predicting at the start of the second innings

columns_to_drop = ['batter', 'bowler', 'non_striker', 'batsman_runs', 'extra_runs',
                   'extras_type', 'player_dismissed', 'dismissal_kind', 'fielder',
                   'has_extras', 'is_wicket_binary', 'match_type', 'player_of_match',
                   'result', 'result_margin', 'target_overs', 'over', 'ball']
# We will re-calculate is_wicket from 'player_dismissed' for first innings.
# 'is_wicket' in your sample is already 0/1 which is good, use that.
# We keep 'total_runs' to calculate 1st innings score.
# We keep 'target_runs' as it seems to be the target set by team1.

In [28]:
df_cleaned = df.drop(columns=[col for col in columns_to_drop if col in df.columns], errors='ignore')

In [29]:
# --- Create Match Summary DataFrame ---

# Group by match_id
match_summary_list = []

for match_id, group in df_cleaned.groupby('match_id'):
    # Get first innings details
    first_inning_df = group[group['inning'] == 1]
    # Get second innings details (to identify chasing team if needed, and actual winner logic)
    second_inning_df = group[group['inning'] == 2]

    if first_inning_df.empty:
        continue # Skip if no first innings data for a match

    # Match-level static info (can be taken from the first row of the match group)
    match_info = group.iloc[0]
    
    team1 = first_inning_df['batting_team'].iloc[0]
    team2 = first_inning_df['bowling_team'].iloc[0] # This is the team that will chase

    # Calculate runs scored and wickets lost in the first innings
    # The 'target_runs' column in your sample is the total score of the team batting first for that match.
    # So we can use that directly. If it wasn't available, we'd sum 'total_runs' for inning 1.
    # Let's verify this assumption or calculate it.
    # If target_runs for inning 1 is the score of team1, let's use it:
    
    # Option 1: Use the provided 'target_runs' if it correctly represents 1st innings total for all matches
    # Ensure it's consistent for all rows of inning 1 for a given match_id
    current_target_runs = first_inning_df['target_runs'].unique()
    if len(current_target_runs) == 1 and current_target_runs[0] > 0 : # check if it's populated for 1st innings
        runs_inning1 = current_target_runs[0]
    else: # fallback to calculate from ball-by-ball
        runs_inning1 = first_inning_df['total_runs'].sum()

    wickets_inning1 = first_inning_df['is_wicket'].sum()

    # Determine if the chasing team (team2) won
    winner = match_info['winner']
    chasing_team_won = 1 if winner == team2 else 0
    
    match_summary_list.append({
        'match_id': match_id,
        'season': match_info['season'],
        'city': match_info['city'],
        'date': match_info['date'],
        'venue': match_info['venue'],
        'team1_bat_first': team1,
        'team2_chase': team2,
        'toss_winner': match_info['toss_winner'],
        'toss_decision': match_info['toss_decision'],
        'target_runs': runs_inning1, # This is the score team2 needs to chase
        'wickets_lost_inn1': wickets_inning1,
        'winner': winner,
        'chasing_team_won': chasing_team_won # Our Target Variable
    })

In [30]:
match_summary_df = pd.DataFrame(match_summary_list)

print("--- Match Summary DataFrame Head ---")
print(match_summary_df.head())
print("\n--- Match Summary DataFrame Info ---")
match_summary_df.info()
print("\n--- Match Summary DataFrame Missing Values ---")
print(match_summary_df.isnull().sum())

--- Match Summary DataFrame Head ---
   match_id  season        city       date  \
0    335982    2008   Bangalore 2008-04-18   
1    335983    2008  Chandigarh 2008-04-19   
2    335984    2008       Delhi 2008-04-19   
3    335985    2008      Mumbai 2008-04-20   
4    335986    2008     Kolkata 2008-04-20   

                                        venue        team1_bat_first  \
0                       M Chinnaswamy Stadium  Kolkata Knight Riders   
1  Punjab Cricket Association Stadium, Mohali    Chennai Super Kings   
2                            Feroz Shah Kotla       Rajasthan Royals   
3                            Wankhede Stadium         Mumbai Indians   
4                                Eden Gardens    Sunrisers Hyderabad   

                   team2_chase                  toss_winner toss_decision  \
0  Royal Challengers Bengaluru  Royal Challengers Bengaluru         field   
1                 Punjab Kings          Chennai Super Kings           bat   
2               Delhi 

In [32]:
# Standardize 'venue' column by removing everything after the first comma
match_summary_df['venue'] = match_summary_df['venue'].str.split(',').str[0].str.strip()


In [33]:
match_summary_df['venue'].unique()

array(['M Chinnaswamy Stadium', 'Punjab Cricket Association Stadium',
       'Feroz Shah Kotla', 'Wankhede Stadium', 'Eden Gardens',
       'Sawai Mansingh Stadium', 'Rajiv Gandhi International Stadium',
       'MA Chidambaram Stadium', 'Dr DY Patil Sports Academy', 'Newlands',
       "St George's Park", 'Kingsmead', 'SuperSport Park', 'Buffalo Park',
       'New Wanderers Stadium', 'De Beers Diamond Oval',
       'OUTsurance Oval', 'Brabourne Stadium', 'Sardar Patel Stadium',
       'Barabati Stadium', 'Vidarbha Cricket Association Stadium',
       'Himachal Pradesh Cricket Association Stadium',
       'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium',
       'Subrata Roy Sahara Stadium',
       'Shaheed Veer Narayan Singh International Stadium',
       'JSCA International Stadium Complex', 'Sheikh Zayed Stadium',
       'Sharjah Cricket Stadium', 'Dubai International Cricket Stadium',
       'Maharashtra Cricket Association Stadium',
       'Punjab Cricket Association IS Bindra S

In [35]:
match_summary_df.to_csv("Match_level_dataset.csv", index=False)